In [25]:
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy.ndimage import *
from load_cppimg import load_img
from MOLLI_fit import *
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")
path = '../exp_result_slow_1700/'
# path = '../exp_result/'
exp_folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
exp_folders.sort(key=lambda x: int(re.search(r'\d+', x).group()))
# print("\n".join(exp_folders))
exp_params=[]
for folder in exp_folders:
    folder_params= re.findall("ID(\d+)_Speed_min(\d+\.\d+)_max(\d+\.\d+)", folder)[0]
    exp_params.append (folder_params)
for fz in zip(exp_folders, exp_params):
    print(fz)

('ID0_Speed_min0.0_max0.01', ('0', '0.0', '0.01'))


In [26]:
def MF_MAGIR_fit(path,show_MOLLI=False,show_T1=False):
    MOLLI_readout_img=[]
    MOLLI_readout_t = np.array( [200, 1200, 2200, 3200, 4200, 300, 1300, 2300]) * 1e-3
    for i in range(8):
        MOLLI_readout_img.append(load_img(
            path,i))
    MOLLI_readout_img = np.asarray(MOLLI_readout_img)

    # print(MOLLI_readout_img[:, 15, 15])
    if show_MOLLI:
        plt.figure(figsize=(18,8))
        for i in range(8):
            plt.subplot(2, 4, i+1)
            plt.imshow(MOLLI_readout_img[i])
            plt.colorbar(shrink=0.7)
        plt.show()

    T1_result = np.zeros(MOLLI_readout_img[0].shape)

    for x, y in tqdm(np.ndindex(MOLLI_readout_img[0].shape)):
        # T1_star, T1, A, B = MAGIR(
        T1_star, T1, A, B = MF_MAGIR(
            MOLLI_readout_t, MOLLI_readout_img[:, x, y])
        T1_result[x, y] = T1
    if show_T1:
        plt.figure()
        plt.imshow(T1_result)
        plt.colorbar()
        plt.clim(0,2)
    return MOLLI_readout_img,T1_result

In [27]:
def get_mask(k=64,x=4,y=4,radius=np.floor(19/256*64)):
    mask_list = []
    for i in range(x):
        for j in range(y):
            mask = np.ones((k,k),dtype=bool)
            for m in range(k):
                for n in range(k):
                    if (m-(i+0.5)*k/x)**2 + (n-(j+0.5)*k/y)**2 < radius**2:
                        mask[m,n] = 0
            mask_list.append(mask)
    return mask_list

In [28]:
# data=[]
# for folder, params in zip(exp_folders, exp_params):
#     print(folder,params)
#     MOLLI_fit, T1_result = MF_MAGIR_fit(os.path.join(path,folder),show_MOLLI=False,show_T1=False)
#     mask_list = get_mask()
#     v_list = np.linspace(start=float(params[1]),stop=float(params[2]),num=16)
#     T1_result_filter = median_filter(T1_result,size=(3,3))
#     for v,mask in zip(v_list,mask_list):
#         mx= np.ma.masked_array(T1_result, mask=mask)
#         print(v,mx.max(),mx.mean())
#         data.append([v,mx.mean()])

# data = np.asarray(data)
# np.save("EXP_Result_SLOW_T1_1500_data.npy",data)

In [29]:
# plt.plot(data[:,0],1000*data[:,1],'x')
# plt.xlabel("Speed (mm/ms)")
# plt.ylabel("T1 (ms)")

In [30]:
mask_list = get_mask()
v_molli_data = []
v_data = []
for folder, params in zip(exp_folders, exp_params):
    print(folder,params)
    MOLLI_readout_img=[]
    for i in range(8):
        MOLLI_readout_img.append(load_img(
            os.path.join(path,folder),i))
    MOLLI_readout_img = np.asarray(MOLLI_readout_img)
    v_list = np.linspace(start=float(params[1]),stop=float(params[2]),num=16)
    v_data.append(v_list)
    for v,mask in zip(v_list,mask_list):
        MOLLI_img_list = []
        for i in range(8):
            MOLLI_img_list.append(np.ma.masked_array(MOLLI_readout_img[i], mask=mask).mean())
        v_molli_data.append(MOLLI_img_list)
v_molli_data=np.asarray(v_molli_data)
v_data = np.hstack(v_data)
print(v_data.shape)
print(v_molli_data.shape)
np.savez("EXP_masked_MOLLI_data_1700.npz",v_data=v_data,molli_data=v_molli_data)

ID0_Speed_min0.0_max0.01 ('0', '0.0', '0.01')
(16,)
(16, 8)
